# 📧 Spam vs Ham — Presentation Notebook

**Cilj:** Kratka, jasna i *demonstrativna* prezentacija projekta u **≤10 minuta**.  
Notebook koristi već pripremljene Python fajlove iz projekta i pravi grafike/ispise koji su potrebni za odbranu.

## Agenda (10 min)
1. **Skup podataka & pretprocesiranje** (1–2 min)  
2. **Klasični modeli (BoW/TF‑IDF)** (3 min)  
3. **Duboki model (DistilBERT)** (3 min)  
4. **Outlier detection (robustnost)** (1 min)  
5. **Uporedni rezultati i zaključak** (1 min)

## 1) Uvoz biblioteka i putanja
Kratan setup: koristimo postojeće module projekta (bez dupliranja koda u notebook-u).

In [ ]:
from pathlib import Path
import json, os, pandas as pd

from src.datasets import load_spamassassin, load_enron, combine_datasets
from src.preprocess import preprocess_series
from src.classical_models import train_classical
from src.deep_models import train_transformer
from src.evaluate import compute_metrics
from src.outlier import anomaly_report

DATA_DIR = Path('data')
OUT_DIR = Path('outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('OK: imports, DATA_DIR =', DATA_DIR.resolve())

## 2) Skup podataka i pretprocesiranje (brzi pregled)
Kratak pregled veličina skupova (bez čitanja velikih fajlova u memoriju).  
Ako je *Enron* veliki, koristi se samo `ham_subset` folder.

In [ ]:
from collections import Counter
import os

def fast_count_spamassassin(root='data/spamassassin'):
    spam = ham = 0
    for dp,_,fns in os.walk(root):
        dp_low = dp.lower()
        if any(x in dp_low for x in ['spam', 'spam_2']):
            spam += len(fns)
        if any(x in dp_low for x in ['ham', 'easy_ham', 'hard_ham', 'legit']):
            ham += len(fns)
    return spam, ham

sa_spam, sa_ham = fast_count_spamassassin()
print(f'SpamAssassin → spam: {sa_spam:,} | ham: {sa_ham:,}')

en_ham = 0
for sub in ['data/enron/ham_subset', 'data/enron/ham']:
    if os.path.isdir(sub):
        for _,_,fns in os.walk(sub):
            en_ham += len(fns)
print(f'Enron → ham: {en_ham:,} (koristi se ham/ham_subset)')

### Pretprocesiranje (primer)
Pokazujemo kako tekst prolazi kroz pipeline čišćenja (lowercase, uklanjanje znakova, lematizacija, `<url>/<email>/<number>` place-holderi).

In [ ]:
samples = pd.Series([
    'WIN $$$ now!!! Click https://promo.example.com and claim your PRIZE!!!',
    'Hi John, meeting moved to 10:30. Send docs to jane.doe@company.com'
])
preprocess_series(samples)

## 3) Klasični modeli (BoW / TF‑IDF)
Kratko treniranje na **kombinovanom** skupu (SpamAssassin + Enron ham_subset).  
Ako je potrebno uštedeti vreme, može se koristiti samo SpamAssassin.

In [ ]:
# Učitaj kombinovani skup (spam iz SpamAssassin + ham iz Enrona)
sa = load_spamassassin('data/spamassassin')
en = load_enron('data/enron')
df = combine_datasets([sa, en])
print(df['label'].value_counts(), '\nTotal:', len(df))

# Treniraj klasične modele na TF-IDF (brže); snimi metrike
report, best_model = train_classical(df['text'], df['label'], features='tfidf', outdir=str(OUT_DIR))
with open(OUT_DIR / 'metrics_classical.json', 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print('Best classical model:', report['best_model_name'], 'F1 =', report['best_f1'])

> **Napomena:** Confusion matrice i ROC krive za ove modele biće snimljene u `outputs/confusion_matrices/` i `outputs/roc_curves/`.

## 4) Duboki model (DistilBERT) — fine-tuning
Ukoliko vreme dozvoljava, može se pustiti kraći fine‑tuning (1–2 epohe).  
Ako je već istreniran ranije, preskoči trening i samo učitaj metrike i prikaži grafikone.

In [ ]:
RUN_TRAIN = False  # ← Postavi na True ako želiš da treniraš tokom prezentacije

if RUN_TRAIN:
    metrics, model_dir = train_transformer(df['text'], df['label'], model_name='distilbert-base-uncased', epochs=1, batch_size=8, outdir=str(OUT_DIR))
else:
    mt_path = OUT_DIR / 'metrics_transformer.json'
    if mt_path.exists():
        with open(mt_path, 'r', encoding='utf-8') as f:
            metrics = json.load(f)
    else:
        metrics = {'eval_accuracy': None, 'eval_f1': None}
metrics

> Za grafike transformera (ROC & Confusion) koristi skriptu:  
```bash
python scripts/eval_deep_plots.py
```
Snimaju se u `outputs/roc_curves/Transformer_roc.png` i `outputs/confusion_matrices/Transformer_confusion.png`.

## 5) Outlier detection (robustnost na nove spam obrasce)
Koristimo **IsolationForest** nad TF‑IDF reprezentacijom da označimo potencijalno anomalne poruke (više = sumnjivije).

In [ ]:
scores = anomaly_report(df['text'], df['label'], features='tfidf', contamination=0.05)
out_path = OUT_DIR / 'outlier_scores.csv'
pd.DataFrame({'text': df['text'], 'label': df['label'], 'anomaly_score': scores}).to_csv(out_path, index=False, encoding='utf-8')
print('Saved:', out_path)

## 6) Uporedni rezultati i grafici
Prikaz uporednih metrika (bar chart) i tabela; dopunjuje ROC krive i confusion matrice.

In [ ]:
!python scripts/plot_summary.py

from IPython.display import Image, display

bar_img = OUT_DIR / 'models_summary_bars.png'
if bar_img.exists():
    display(Image(filename=str(bar_img)))

for p in [
    OUT_DIR / 'roc_curves' / 'LinearSVM_roc.png',
    OUT_DIR / 'roc_curves' / 'LogReg_roc.png',
    OUT_DIR / 'roc_curves' / 'NaiveBayes_roc.png',
    OUT_DIR / 'roc_curves' / 'Transformer_roc.png',
    OUT_DIR / 'confusion_matrices' / 'LinearSVM_confusion.png',
    OUT_DIR / 'confusion_matrices' / 'LogReg_confusion.png',
    OUT_DIR / 'confusion_matrices' / 'NaiveBayes_confusion.png',
    OUT_DIR / 'confusion_matrices' / 'Transformer_confusion.png',
]:
    if p.exists():
        display(Image(filename=str(p)))

## 7) Zaključak
- **Pretprocesiranje**: čišćenje, normalizacija specifičnih email tokena, lematizacija.  
- **Modeli**: NB/LR/LinearSVM (klasični) i DistilBERT (duboki) — svi trenirani i evaluirani.  
- **Metrike**: accuracy, precision, recall, F1, ROC-AUC; fokus na *spam* kao pozitivnu klasu.  
- **Robustnost**: detekcija outliera (IsolationForest) za nove/neviđene spam obrasce.  
- **Rezultat**: odabrati model sa najboljim **F1** i praktičnim performansama (u tvom slučaju, **LinearSVM** ili **Transformer**).